# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [12]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [20]:
!ollama pull llama3.2

]11;?\pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  92% ▕████████████████  ▏ 1.9 GB/2.0 GB        

In [23]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [24]:

cloud_llm = OpenAI()
local_llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [26]:
TUTOR_INSTRUCTIONS = """\
You are a helpful coding mentor. Your job is to break down programming topics \
so they are easy to follow for someone learning to code.

Format every answer like this:
1. **Quick summary** — one or two sentences capturing the core idea.
2. **Detailed walkthrough** — explain step by step with runnable code snippets.
3. **Watch-outs** — list 2-3 traps that trip up newcomers.

Use real-world analogies whenever they make a concept more intuitive.

Below are two sample answers so you know exactly what style to follow.

=== Sample A ===

Topic: How do Python decorators work?

**Quick summary**
A decorator is a function that wraps another function to extend its behaviour \
without modifying the original code.

**Detailed walkthrough**
At its simplest, a decorator takes a function as input and returns a new function:

    def shout(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            return result.upper()
        return wrapper

    @shout
    def greet(name):
        return f"hello {name}"

    greet("world")  # "HELLO WORLD"

The `@shout` syntax is shorthand for `greet = shout(greet)`. \
Think of it like putting a phone case on your phone — the phone (function) \
still works the same, but the case (decorator) adds extra protection or style.

**Watch-outs**
- Forgetting `@functools.wraps(func)` inside the wrapper — without it the \
original function's name and docstring get hidden.
- Stacking multiple decorators without understanding the order — they apply \
bottom-up, not top-down.
- Trying to decorate a function that relies on its own identity (e.g. recursion \
by name) can behave unexpectedly.

=== Sample B ===

Topic: What is the difference between `==` and `is` in Python?

**Quick summary**
`==` checks whether two objects have the same *value*; `is` checks whether they \
are the exact *same object* in memory.

**Detailed walkthrough**

    a = [1, 2, 3]
    b = [1, 2, 3]

    a == b   # True  — same contents
    a is b   # False — two separate list objects

    c = a
    a is c   # True  — c points to the same list as a

Analogy: two identical copies of a book have equal content (`==`), but they are \
not the same physical book (`is`).

Python caches small integers (-5 to 256) and short strings, so `is` can \
*accidentally* return True for those — never rely on it for value comparison.

**Watch-outs**
- Using `is` to compare strings or numbers — works sometimes due to caching, \
fails unpredictably with larger values.
- Confusing `is not` with `!=` — they test different things.
- Mutating a list through an alias (`c.append(4)` also changes `a`) because \
`is` tells you they share identity.
"""

In [ ]:
def ask_ai_tutor(topic: str, *, model: str, use_local: bool = False):
    """Stream a mentor-style explanation for *topic*.

    Set use_local=True to run against Ollama.
    """
    provider = local_llm if use_local else cloud_llm
    tag = f"local ({model})" if use_local else f"cloud ({model})"

    prompt = f"Please explain the following coding topic using the format from your instructions.\n\nTopic: {topic}"

    try:
        chunks = provider.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": TUTOR_INSTRUCTIONS},
                {"role": "user", "content": prompt},
            ],
            stream=True,
        )
    except Exception as err:
        display(Markdown(f"**Could not reach {tag}:** {err}"))
        return

    answer = ""
    output_handle = display(Markdown(""), display_id=True)
    for part in chunks:
        token = part.choices[0].delta.content or ""
        answer += token
        update_display(Markdown(answer), display_id=output_handle.display_id)

In [28]:
my_question = "What does the 'yield from' statement do in Python and when should I use it?"

### GPT-4.1-mini

In [29]:
ask_ai_tutor(my_question, model="gpt-4.1-mini")

**Quick summary**  
`yield from` is a shortcut for yielding all values from a sub-generator or iterable, delegating part of a generator’s operations to another generator. It makes generator code cleaner and enables seamless composition of generators.

**Detailed walkthrough**  
Imagine you’re listening to a playlist of songs (values) but one song is actually another entire playlist (sub-generator). Instead of listening to that playlist separately, `yield from` lets you play all songs from the sub-playlist directly, one after another.

Here’s a basic example without `yield from`:

```python
def generator1():
    yield 1
    yield 2

def generator2():
    yield 'a'
    yield 'b'

def combined():
    # Manually yield from generator1
    for value in generator1():
        yield value
    # Manually yield from generator2
    for value in generator2():
        yield value

for item in combined():
    print(item)
```

Output:  
```
1
2
a
b
```

Now using `yield from` simplifies the combined generator:

```python
def combined():
    yield from generator1()
    yield from generator2()

for item in combined():
    print(item)
```

This does exactly the same but is cleaner and easier to read.

A more advanced aspect is that `yield from` also forwards values sent to the outer generator and exceptions thrown into the sub-generator, and returns the sub-generator’s return value (available in Python 3.3+).

Example demonstrating capturing the return value of a sub-generator:

```python
def sub_gen():
    yield 1
    return 'done'

def main_gen():
    result = yield from sub_gen()
    print(f"Sub-generator returned: {result}")

for _ in main_gen():
    pass
```

Output:  
```
Sub-generator returned: done
```

This kind of delegation is very helpful in complex generator pipelines or coroutine-style code, where one generator controls or wraps another.

**Watch-outs**  
- Trying to use `yield from` with something that is not iterable (like an int or None) will raise a `TypeError`. It must be used on iterators or generators.  
- Remember `yield from` replaces a typical `for…yield` loop; mixing the two styles inside the same generator can be confusing.  
- The `yield from` syntax is only available in Python 3.3 and later — older Python versions require manual iteration.  

Think of `yield from` like handing over the baton smoothly in a relay race instead of stopping and starting multiple times yourself. It keeps your generator logic clean and efficient.

### Llama 3.2 (via Ollama)

In [30]:
ask_ai_tutor(my_question, model="llama3.2", use_local=True)

**Quick summary**
`yield from` is a special keyword that allows one generator to delegate part of its execution to another iterable (like a list, tuple or other generator), deferring the responsibility and avoiding potential memory issues.

**Detailed walkthrough**

Imagine you have two generators: `outer` and `inner`. The `outer` generator needs to iterate over some data but also wants to break down that task into smaller chunks. If it were to do that manually using `for` loops, it would load all the intermediate results in memory at once (which isn't necessary and can be counterproductive for large datasets).

That's where `yield from` comes in:

```python
def outer():
    yield "Start"
    
    # `yield from inner()` breaks down the long task into smaller parts:
    for value in inner():
        print(f"Received value: {value}")
        
    yield "End"

def inner():
    for i in range(5):
        yield f"Inner {i}"
```

Running this as a single generator:

```python
for item in outer():
    print(item)
# Output:
# Start
# Received value: Inner 0
# Received value: Inner 1
# Received value: Inner 2
# Received value: Inner 3
# Received value: Inner 4
# End
```

This approach shows how `yield from` can save memory because only the next inner value is stored in memory at a time. If we were to fetch all inner values first using some other method, we'd have unnecessary extra space (memory) usage.

**Watch-outs**

- Using `yield from` unnecessarily consumes unnecessary resources. There's a point of diminishing return past which benefits of this optimization don't outweigh the complexity.
- In rare cases when iterating over an iterator created by `yield`, some backtracking might be problematic due to its internal state, e.g., tracking how many items have been printed so far (due to `yield from...while` loop iterations).